In [20]:
import psycopg2
from sklearn import svm
import pandas.io.sql as psql
import psycopg2
import re
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import grid_search
import xgboost as xgb
from sklearn.externals import joblib
from sklearn import svm
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [2]:
import json
with open('/Users/krishna/MOOC/smart-city/login.json') as data_file:
    db = json.load(data_file)

conn = psycopg2.connect(database=db['dbname'], user=db['user'], password=db['password'], host=db['host'], port=db['port'])

In [3]:
dataframe = psql.frame_query("select idd, text, 2*alch_score +3 as alch_score_norm, local_score from twitter.tweets where (2*alch_score+3) - local_score < 1 and (2*alch_score+3 - local_score) > - 1 and alch_score != '0'  ;", conn)

/usr/local/lib/python2.7/site-packages/pandas/io/sql.py:1569: FutureWarning: frame_query is deprecated, use read_sql
  warnings.warn("frame_query is deprecated, use read_sql", FutureWarning)


In [4]:
dataframe.shape

(9599, 4)

In [6]:
dataframe['avg'] = dataframe.apply(lambda x : round((x['alch_score_norm'] + x['local_score'])/2) ,axis=1)

In [7]:
train_data, test_data = train_test_split(dataframe, test_size = 0.2)

In [12]:
print "Size of training data", len(train_data)
print "Size of test data",len(test_data)

Size of training data 7679
Size of test data 1920


In [5]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [13]:
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 1.0,
                             sublinear_tf=True,
                             use_idf=True)
train_vectors = vectorizer.fit_transform(train_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_vectors, train_data['avg'])

In [22]:
y_predicted = clf.predict(test_vectors)
y_test = test_data['avg']

In [23]:
print metrics.classification_report(y_test, y_predicted)

             precision    recall  f1-score   support

        1.0       1.00      0.12      0.22        48
        2.0       0.63      0.80      0.71       811
        3.0       0.59      0.59      0.59       588
        4.0       0.75      0.49      0.59       457
        5.0       0.00      0.00      0.00        16

avg / total       0.65      0.64      0.63      1920



/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [56]:
parameters = {'kernel':('linear', 'rbf')}
svr = svm.SVC()
clf10 = grid_search.GridSearchCV(svr, parameters)
clf10.fit(train_vectors, train_data['avg'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf')}, pre_dispatch='2*n_jobs',
       refit=True, scoring=None, verbose=0)

In [57]:
y_predicted1 = clf10.predict(test_vectors)

In [58]:
print metrics.classification_report(y_test, y_predicted1)

             precision    recall  f1-score   support

        1.0       1.00      0.29      0.45        48
        2.0       0.74      0.82      0.78       811
        3.0       0.67      0.68      0.67       588
        4.0       0.74      0.67      0.70       457
        5.0       0.00      0.00      0.00        16

avg / total       0.72      0.72      0.71      1920



In [55]:
clf.best_params_

{'kernel': 'linear'}

In [52]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier 
clf1 = RandomForestClassifier(n_estimators=50)
forest = clf1.fit(train_vectors, train_data['avg'])

In [53]:
y_predicted2 = clf1.predict(test_vectors)

In [54]:
print metrics.classification_report(y_test, y_predicted2)

             precision    recall  f1-score   support

        1.0       1.00      0.44      0.61        48
        2.0       0.70      0.82      0.76       811
        3.0       0.66      0.70      0.68       588
        4.0       0.83      0.58      0.68       457
        5.0       0.00      0.00      0.00        16

avg / total       0.72      0.71      0.70      1920



Export SVM model to production

In [59]:
joblib.dump(clf10, "ModelObjects/modelsvm07June.pkl") 
joblib.dump(vectorizer, "ModelObjects/tfidf07June.pkl") 

['ModelObjects/tfidf07June.pkl',
 'ModelObjects/tfidf07June.pkl_01.npy',
 'ModelObjects/tfidf07June.pkl_02.npy']